# U4 Data Analysis
## M02 Demo - Anomaly Detection Wrangling Script

Previous code from demos

In [12]:
# Load the required packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import datetime as dt
# Set global options
pd.set_option('display.float_format', '{:.2f}'.format) 

## Data Import

In [14]:
# Load in the data - Change to the users file locations
# Import the journals dataset as journals
journals=pd.read_excel('Journals2.xlsx')
# Import the bank holidays dataset as bank_hols 
bank_hols = pd.read_excel('Bank Holidays 2018-2020.xlsx')

## Data Wrangling

Change data types

In [16]:
# Specify date formal
journals['JnlDateTime'] = pd.to_datetime(journals['JnlDateTime'], format = '%d/%m/%Y %H:%M')
journals['JnlDate']=journals['JnlDateTime'].dt.date
# Specify JnlNo is a string
journals.JnlNo = journals.JnlNo.astype('str')

### Add new columns for analysis

Add new time columns

In [21]:
# Create columns for time, hour and date using the datetime command dt 
journals['JnlTime'] = journals['JnlDateTime'].dt.time              
journals['Hr'] = journals['JnlDateTime'].dt.hour

Add continuous time variable for the time of day journal posted

In [23]:
# Add the hour to the minutes as a fraction of an hour to create a continuous number
journals['JnlTimeN']=journals['Hr']+journals['JnlDateTime'].dt.minute/60

Add continuous variable for day of the month journal posted

In [25]:
# Create a column for Day of the Month journal is posted
journals['DoM']=journals['JnlDateTime'].dt.day

Add a column for absolute amount

In [27]:
# Absolute the amount value using the abs command in the numpy package
journals['AbsAmount'] = np.abs(journals['Amount']) 

Add a column with total number of transaction per the journal number

In [29]:
# Calculate the total number of transaction by grouping by journal number and selecting a column
NTrans=pd.DataFrame({'NTrans':journals.groupby('JnlNo').count().Account})
# Add this column to the journals dataset
journals=journals.merge(NTrans,left_on='JnlNo',right_on='JnlNo',how="left")

## Create True and False Tests

1. Preparer is the same as Authorisor

In [32]:
# Create a new column which evaluates if journal preparer is identical to journal authoriser
journals = journals.eval('AuthIsRaiser = JnlPrep == JnlAuth')
# Change from boolean to numeric
journals['AuthIsRaiser'] = journals['AuthIsRaiser'].astype(int) 

2. Journal Description is Blank

In [35]:
# Create a new column that checks if journal description is missing
journals['JnlDescBlank'] = journals['JnlDesc'].isna()
# Change from boolean to numeric
journals['JnlDescBlank'] = journals['JnlDescBlank'].astype(int) 

3. Journals Posted Out-of-hours

In [39]:
# Create a new column that evaluates if time posted is out of working hours (set as 8am til 6pm)
journals['JnlOOH'] = journals['Hr'].apply(lambda x: 1 if x>=18 or x<=8 else 0)

4. Journals Posted at weekend

In [42]:
# Creates a day of the week column using datetime (this is represented in numeric form 0-6)
journals['JnlDoW'] = journals['JnlDateTime'].dt.dayofweek
# Creates a new column that evaluates if journal is posted at a weekend (i.e 5 or 6)
journals['AtWeekend'] = journals['JnlDoW'].apply(lambda x: 1 if x > 4 else 0)

5. Unbalanced Journals

In [50]:
# Creates a subset of journal indexes whose total debit and credits does not balance
unbalancedjnls = journals[['JnlNo','Credit','Debit']]
unbalancedjnls = unbalancedjnls.groupby('JnlNo').sum().eval('JnlDiff = Debit-Credit').query('JnlDiff != 0.00').index
# Creates a column that indicates whether each JnlNo is in the unbalanced journals subset
journals['Unbalanced'] = journals['JnlNo'].apply(lambda x: 1 if x in unbalancedjnls else 0)

6. Journals Posted on a Holiday

In [52]:
# Set the relevant date columns to strings for comparison 
journals['DateMerge']=journals['JnlDate'].astype(str)
bank_hols['Holiday']=bank_hols['Holiday'].astype(str)
# Merge the datasets to obtain the holiday date added in
journals=journals.merge(bank_hols, right_on="Holiday",left_on="DateMerge",how="left")
# Create a new column that says whether there is a holiday date given or not
journals['IsHoliday'] = journals["Holiday"].isna().apply(lambda x: 1 if x==0 else 0)

## Final Dataset Prep

Combine Truth and False into a continuous score

In [55]:
# Sum the true or falses to obtain total score
journals = journals.eval('TotScore = AuthIsRaiser + JnlDescBlank + JnlOOH + AtWeekend + Unbalanced + IsHoliday')

Select columns for PCA

In [57]:
journalsprePCA=journals[["JnlNo","NTrans","DoM","JnlTimeN","TotScore"]]

Combine with total amount for journals

In [59]:
# Group by Journal number and sum and half the Absolute amount to calculate the total value
journalstotals=journals.groupby("JnlNo")["AbsAmount"].sum()/2
# Turn this into a dataframe and merge to obtain details for each journal number
journalstotals=pd.DataFrame(journalstotals)
journalsprePCA=journalstotals.merge(journalsprePCA,how="left",right_on="JnlNo",left_on="JnlNo").drop_duplicates().set_index("JnlNo")

In [61]:
journalsprePCA.head()

,AbsAmount,NTrans,DoM,JnlTimeN,TotScore
JnlNo,,,,,
1,300859.89,4,1,13.07,1
10,12848.50,2,30,9.93,0
100,24839.41,2,30,16.92,1
101,22257.21,6,30,10.98,2
102,785539.06,4,30,9.38,1
